In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sys
from sklearn.neighbors import NearestNeighbors

In [ ]:
# function to calculate the distance between two points
def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# function to calculate the value factor of a bank: money/(rob time + travel time), return both the time spent at this back (including travel time) and the value_factor of this back
def value_factor(money, time, distance):
    ttime = time + distance/30
    value = money/ttime
    return (ttime, value)

In [ ]:
def robber_algorithm(df):
    # Create a list of all banks' coordinates (x,y)
    points = list(zip(df['x_coordinate'],df['y_coordinate']))
    # Create a knn model to fit all the points
    knn = NearestNeighbors(n_neighbors = 500).fit(points)

    prev = (0,0) # start from the helicopter zone (0,0) and go backwards
    path = [] # Create an empty list for the travel path
    time_left = 24 # The time limit

    while True:
        values = [] # An empty list to store the values
        time_spent = [] # An empty list to store the time needed to rob this bank including travel time
        # The k closest neighbors for the current point
        nbrs = knn.kneighbors([prev], return_distance = False)[0]
        for row in nbrs:
            if row not in path:
                # calculate the distance between this point and its closest neighbor
                x1, y1 = points[row]
                x2, y2 = prev
                dist = distance(x1, y1, x2, y2)
                # calculate the value_factor of each of the neighbors
                ttime, value = value_factor(df.iloc[row]['money'], df.iloc[row]['time (hr)'], dist)
                values.append(value)
                time_spent.append(ttime)
            else:
                values.append(0)
                time_spent.append(0)
        
        # pick the bank with the highest value factor
        ind = values.index(max(values))
        step=nbrs[ind]
        # if there is still time left to rob this bank, add it to the path, otherwise, stop here
        if time_left - time_spent[ind] >=0:
            path.append(step)
            time_left -= time_spent[ind]
            prev = points[step]
        else:
            break
    
    my_path = path[::-1]
    return my_path